---
title: "18-SAS 拟合优度检验"
author: "Simon Zhou"
date: "2025-06-10"
date-modified: "2025-06-10"
format: 
    html:
        code-fold: true
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [1]:
%load_ext saspy.sas_magic

# 拟合优度检验

医学研究中常需推断某现象的频数分布是否服从某一理论分布,如正态性检验就是推断某资料是否服从正态分布的一种检验方法，而且只适用于正态分布。而要推断资料是否服从二项分布、Poisson分布或负二项分布等则需用到检验。

因为 Pearsonx值能反映实际频数和理论频数的吻合程度,所以，检验可用于推断频数分布的拟合优度(Goodness offit )。

## 示例

观察某克山病区克山病患者的空间分布情况。调查者将该地区划分为279个取样单位,统计各取样单位历年累计病例数,资料见下表。问此资料是否服从 Poisson 分布?

|取样单位内病例数|观察频数|
| ---- | ---- |
|0|26|
|1|51|
|2|75|
|3|63|
|4|38|
|5|17|
|6|5|
|7|3|
|≥8|1|
|合计|279|

In [3]:
%%SAS
/*程序18-1*/
data prg9_12;
    input x f @@;
    t= x*f;
datalines;
0 26 1 51 2 75 3 63 4 38 5 17 6 5 7 3 8 1
;
run;
proc means sum noprint;
    var t f;
output out = b sum = sumt sumf;
run;
data c;
    set b;
        do x=0 to 8;
            lamda=sumt/sumf;
            output;
        end;
run;
data d;
    merge prg9_12 c;
    by x;
run;
data e;
    set d;
    if x = 0 then p = poisson(lamda,0);
    if x > 0 and x < 8 then p = poisson(lamda,x)- poisson(lamda,x-1);
    if x = 8 then p = 1-poisson(amda,x-1);
    retain chisq o p 0;
    t1 = sumf*p;
    chisq =((f-t1)**2)/t1;
run;
proc means sum noprint;
    var chisq;
    output out =f sum = sumchi;
run;
data g;
    set f;
    p_chi= 1-probchi(sumchi,8);
run;
proc print;
    var sumchi p_chi;
run;

观测,sumchi,p_chi
1,2.48975,0.96221


### 程序说明

这里的程序有点复杂，主要可以拆分为 7 个步骤：

1. 首先建立数据集 prg9_12,其中拥有两个变量 x 和 f ,变量 x 表示取样单位内的病例数,变量 f 为发生病例数的频数。另外产生一个变量,该变量表示取样单位内的总发病人数。
2. 然后对该数据集用 `means` 过程计算总发病人数和总人数,并将这两个统计量输出到数据集 b 中。
3. 创建数据集 c ,调用数据集 b ,计算总发病率(lamda),并用 `do-end` 语句产生 x 变量,该变量的值从 0 到 8 ,所以此时数据集中有四个变量:总发病人数(sumt)、总人数(sumf)、总发病率(lamda)和x(单位病例数),观测数有9例这9例观测变量x的值从 0 到 8,其他变量值都相同。
4. 将数据集 prg9_12 和 c 以 x 为关键变量进行合并,产生数据集 d。再建数据集 e,调用数据集 d,根据数据集 d 的变量x的数值产生每个 x 值的 Poisson 的概率值(P)。
5. 然后通过 Poisson 的概率值计算每个单位病例数的理论发病人数(t),从而计算出各单位病例数所对应的 x 值( chisq )。
6. 用 `means` 过程计算，值的总和( sumchi )。并将结果输出到数据集 f。
7. 另建立数据集 g,调用数据集 f,根据 X 值的合计,用 `probchi` 函数计算出该 $\chi^2$ 值所对应的 P 值(p_chi),最后将结果输出到 Outpu 窗口。

### 结果说明

本例 $\chi^2=2.48975,P=0.96221>0.05$,说明该克山病区克山病患者的空间分布服从 Poisson 分布.